In [2]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from nirs4all.transformations import StandardNormalVariate as SNV, SavitzkyGolay as SG, Gaussian as GS
from nirs4all.transformations import Rotate_Translate as RT

from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold, ShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

import json
from sample import config as python_config


In [20]:
%load_ext autoreload
%autoreload 2

from SpectraDataset import SpectraDataset
from PipelineRunner import PipelineRunner


# Load dataset (using current SpectraDataset API)
dataset_py = SpectraDataset.from_config(python_config)
dataset_json = SpectraDataset.from_config("sample.json")
dataset_yaml = SpectraDataset.from_config("sample.yaml")

print("\n", "="*200, "\nPython Dataset:\n", dataset_py)
print("\n", "="*200, "\nJSON Dataset:\n", dataset_json)
print("\n", "="*200, "\nYAML Dataset:\n", dataset_yaml)

# Execute with different config types

runner = PipelineRunner(max_workers=4, continue_on_error=True)
history_py = runner.run(python_config, dataset_py)
history_json = runner.run("sample.json", dataset_json)
history_yaml = runner.run("sample.yaml", dataset_yaml)

# # Get execution summary
# summary = runner.get_execution_summary()
# print(f"Executed {summary['total_steps']} steps")
# print(f"Success rate: {summary['successful_steps']}/{summary['total_steps']}")

# # Access predictions
# predictions = summary['predictions']
# for model_name, preds in predictions.items():
#     print(f"Model {model_name}: {preds.shape}")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'dataset': {'type': 'classification', 'folder': './sample_data'}, 'pipeline': ['PlotModelPerformance', MinMaxScaler(), 'PlotModelPerformance', {'feature_augmentation': [None, <class 'nirs4all.transformations._nirs.SavitzkyGolay'>, [<class 'sklearn.preprocessing._data.StandardScaler'>, <class 'nirs4all.transformations._standard.Gaussian'>]]}, 'PlotModelPerformance', {'sample_augmentation': [<class 'nirs4all.transformations._random_augmentation.Rotate_Translate'>, Rotate_Translate(p_range=3)]}, 'PlotModelPerformance', ShuffleSplit(n_splits=10, random_state=None, test_size=None, train_size=None), 'PlotModelPerformance', {'cluster': KMeans(n_clusters=5, random_state=42)}, 'PlotModelPerformance', RepeatedStratifiedKFold(n_repeats=2, n_splits=5, random_state=42), 'PlotModelPerformance', 'uncluster', 'PlotData', {'dispatch': [[MinMaxScaler(), {'feature_augmentation': [None, <class 'nirs4all.transformation

### Preparation Tests

In [8]:
%load_ext autoreload
%autoreload 2

# Example 1: Single dataset configuration
single_config = {
    "dataset": {
        "X": "./sample_data/Xcal.csv",
        "Y": {"from": 0, "to": 3},
        "params": {
            "delimiter": ";",
            "decimal": ".",
            "na_policy": "auto"
        }
    }
}

# Example 2: Multiple datasets configuration
multi_config = {
    "dataset": {
        "train": {
            "X": "./sample_data/Xcal.csv",
            "Y": "./sample_data/Ycal.csv",
        },
        "test": {
            "X": "./sample_data/Xval.csv",
            "Y": "./sample_data/Yval.csv",
        },
        # "valid": {
        #     "X": "/path/to/valid_features.csv",
        #     "Y": [0, 1, 2]
        # }
    }
}

# Example 3: Folder configuration
folder_config = {
    "dataset": "./sample_data/"
}

from spectra.CsvLoader import load_data_from_config

try:
    print("Data loader functions ready to use!")

    print("# For single dataset:")
    X, Y = load_data_from_config(single_config)
    print(f"Loaded single dataset: X shape {X.shape}, Y shape {Y.shape}")

    print("\n# For multiple datasets:")
    datasets = load_data_from_config(multi_config)
    for name, (X_data, Y_data) in datasets.items():
        print(f"Loaded {name}: X shape {X_data.shape}, Y shape {Y_data.shape}")

    print("\n# For folder data:")
    X, Y = load_data_from_config(folder_config)
    print(f"Loaded folder data: X shape {X.shape}, Y shape {Y.shape}")

    print(type(Y[0]))

except Exception as e:
    print(f"Example failed (expected with dummy paths): {e}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Data loader functions ready to use!
# For single dataset:
Loading single XY dataset...
Loaded single dataset: X shape (130, 2148), Y shape (130, 3)

# For multiple datasets:
Loading multiple datasets...
Loaded train: X(130, 2151), Y(130, 1)
Loaded test: X(59, 2151), Y(59, 1)
Loaded train: X shape (130, 2151), Y shape (130, 1)
Loaded test: X shape (59, 2151), Y shape (59, 1)

# For folder data:
Loading data from folder structure...
Loaded folder data: X shape (130, 2151), Y shape (130, 1)
<class 'numpy.ndarray'>
